# 서울특별시 다산콜센터(☎120)의 주요 민원
* 서울특별시 다산콜센터(☎120)의 주요 민원(자주 묻는 질문)에 대한 답변정보
* https://opengov.seoul.go.kr/civilappeal/list

In [2]:
# 필요한 도구를 불러온다.
# 파이썬에서 사용할 수 있는 엑셀과 유사한 데이터분석 도구
import pandas as pd
# 매우 작은 브라우저로 웹사이트의 내용과 정보를 불러옴
import requests
# request로 가져온 웹사이트의 html 태그를 파싱하는데 사용
from bs4 import BeautifulSoup as bs
import time
# 대량 데이터 처리시 진행 상황을 표시
from tqdm import tqdm, trange
# 정규표현식
import re

In [5]:
# 120 다산 콜센터의 첫 페이지를 먼저 불러와 크롤링할 내용을 봅니다.
base_url = "https://opengov.seoul.go.kr/civilappeal/list?items_per_page=1000&"

In [6]:
# pd.read_htm 을 통해 해당 URL의 table 정보를 읽어옵니다.
table = pd.read_html(base_url, encoding="utf8")[0]
table

,번호,제목,생산일,조회수
0,2668,아빠 육아휴직 장려금,2021-08-17,130
1,2667,[서울산업진흥원] 서울메이드란?,2021-06-29,163
2,2666,(강북구) 정비중,2021-06-02,212
3,2665,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)",2021-05-13,286
4,2664,마포 뇌병변장애인 비전센터,2021-03-12,438
...,...,...,...,...
995,1673,새마을운동조직의 지원근거법령은?,2009-10-16,195
996,1672,2 이상의 토지나 건축물이 있는 경우 매매를 할 경우 별도의 분양대상이 되는지요?,2009-10-16,324
997,1671,은평구에서의 사회단체보조금 지원대상 사업은?,2009-10-16,142
998,1670,2 이상의 주택을 가지고 있는 경우 아파트 분양대상은 몇 개인가요?,2009-10-16,356


## 상세정보를 위한 링크정보 수집

In [4]:
# 웹페이지의 결과를 받아옵니다.
response = requests.get( base_url )
# html 태그를 파싱해 올 수 있도록 합니다.
html = bs(response.text, 'html.parser')

In [5]:
# 응답코드 확인 200 == OK
response.status_code

200

## 상세 정보 수집을 위한 링크 정보 찾기

In [6]:
# BeautifulSoup 의 select 기능을 통한 링크 태그 찾기
a_list = html.select("td > a")
a_link_no = [a["href"].replace("/civilappeal/", "") for a in a_list]

In [7]:
table["view_no"] = a_link_no
table

,번호,제목,생산일,조회수,view_no
0,2668,아빠 육아휴직 장려금,2021-08-17,118,23522464
1,2667,[서울산업진흥원] 서울메이드란?,2021-06-29,158,23194045
2,2666,(강북구) 정비중,2021-06-02,209,23032485
3,2665,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)",2021-05-13,279,22904492
4,2664,마포 뇌병변장애인 비전센터,2021-03-12,435,22477798
5,2663,2021년도 중1·고1 신입생 입학준비금 지원,2021-02-09,266,22227896
6,2662,[동대문구]반려견 놀이터,2020-12-08,638,21780668
7,2661,위드유 서울 직장 성희롱.성폭력 예방센터,2020-09-16,762,21212235
8,2660,"[성동구] 반려동물 ""무료상담실"" 운영 안내",2020-08-31,498,21095911
9,2659,서울시 '우리동네꼼꼼육아정보' 책자,2019-10-23,1170,18955676


## 특정 페이지를 수집하는 함수 만들기

In [8]:
def get_one_page(page_no):
    base_url = f"https://opengov.seoul.go.kr/civilappeal/list?items_per_page=1000&page={page_no}"
    table = pd.read_html(base_url, encoding="utf8")[0]
    # 웹페이지의 결과를 받아온다.
    response = requests.get( base_url )
    # html 태그를 파싱해 올 수 있도록 한다.
    try:
        html = bs(response.text, 'html.parser')
        a_list = html.select("td > a")
        a_link_no = [a["href"].replace("/civilappeal/", "") for a in a_list]
        table["view_no"] = a_link_no
        return table
    except:
        return pd.DataFrame()

In [9]:
# 함수가 잘 동작하는지 확인
get_one_page(1)

,번호,제목,생산일,조회수,view_no
0,2668,아빠 육아휴직 장려금,2021-08-17,118,23522464
1,2667,[서울산업진흥원] 서울메이드란?,2021-06-29,158,23194045
2,2666,(강북구) 정비중,2021-06-02,209,23032485
3,2665,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)",2021-05-13,279,22904492
4,2664,마포 뇌병변장애인 비전센터,2021-03-12,435,22477798
5,2663,2021년도 중1·고1 신입생 입학준비금 지원,2021-02-09,266,22227896
6,2662,[동대문구]반려견 놀이터,2020-12-08,638,21780668
7,2661,위드유 서울 직장 성희롱.성폭력 예방센터,2020-09-16,762,21212235
8,2660,"[성동구] 반려동물 ""무료상담실"" 운영 안내",2020-08-31,498,21095911
9,2659,서울시 '우리동네꼼꼼육아정보' 책자,2019-10-23,1170,18955676


In [10]:
# 없는 페이지도 확인
get_one_page(10000)

""


## 반복문을 통한 여러 페이지 수집하기

* 게시물이 없을 때까지 수집하기

In [11]:
# 0.01 초씩 쉬었다가 가져옴
# 게시물이 없으면 멈춤
page_no = 1
table_list = []
while True:
    table = get_one_page(page_no)
    table_list.append(table)
    page_no = page_no + 1
    time.sleep(0.01)
    if table.empty:
        break

## 데이터 병합하기

In [12]:
# 여러 페이지의 내용을 하나로 병합
df = pd.concat(table_list)

## 데이터 파일로 저장하기

In [ ]:
file_name = "seoul-120-list.csv"

In [13]:
df.to_csv(file_name, index=False)

In [14]:
pd.read_csv(file_name)

,번호,제목,생산일,조회수,view_no
0,2668,아빠 육아휴직 장려금,2021-08-17,118,23522464
1,2667,[서울산업진흥원] 서울메이드란?,2021-06-29,158,23194045
2,2666,(강북구) 정비중,2021-06-02,209,23032485
3,2665,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)",2021-05-13,279,22904492
4,2664,마포 뇌병변장애인 비전센터,2021-03-12,435,22477798
...,...,...,...,...,...
2663,5,홈페이지를 구축해서 운영을 하려고 하는데 서버를 어떻게 사용할 수 있나요?,2007-01-07,257,2894434
2664,4,수돗물의 톤당 원가는?,2007-01-07,227,19353862
2665,3,출산휴가후 집근처에 안심하고 맡길 수 있는 어린이집이 있는지?,2007-01-07,202,19699584
2666,2,자동차검사장을 지정받고자 하는데 어떻게 해야 하나요?,2007-01-07,176,2898293
